<a href="https://colab.research.google.com/github/JuanPabl07DP/SimilacionSD/blob/main/Simulaci%C3%B3nSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación de bot para el monitoreo de emociones (Prototipo)

## Desarrollado por:
- Juan Pablo Daza Pereira
- Juan Sebastian Camargo Sanchez

In [1]:
from google.colab import files
!pip install boto3
import boto3
import os
import sys
import zipfile
import subprocess
import wave
import numpy as np
import ctypes
from collections import defaultdict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.2 MB/s eta 0:00:00


In [2]:
print("🔼 Sube una imagen (con rostro):")
uploaded_img = files.upload()

# Obtener el nombre del archivo
image_file = list(uploaded_img.keys())[0]

# Leer imagen como bytes
with open(image_file, 'rb') as image:
    image_bytes = image.read()

#Para audio
print("🔼 Sube un archivo de audio (.wav o .mp3):")
uploaded_audio = files.upload()

audio_file = list(uploaded_audio.keys())[0]

🔼 Sube una imagen (con rostro):


Saving D7JCL4RM3BB5TMC4NNRYOJQADA.jpg to D7JCL4RM3BB5TMC4NNRYOJQADA.jpg
🔼 Sube un archivo de audio (.wav o .mp3):


Saving record_out (1).wav to record_out (1).wav


In [3]:
AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = ""
AWS_REGION = ""

In [4]:
class FusionEmociones:
    def __init__(self):
        # Mapeo entre emociones de AWS Rekognition y Vokaturi
        self.mapeo_emociones = {
            'HAPPY': 'Felicidad',
            'SAD': 'Tristeza',
            'ANGRY': 'Ira',
            'FEAR': 'Miedo',
            'CALM': 'Neutral',
            'SURPRISED': None,
            'DISGUSTED': None,
            'CONFUSED': None
        }

        # Emociones comunes entre ambos sistemas
        self.emociones_comunes = ['Felicidad', 'Tristeza', 'Ira', 'Miedo', 'Neutral']

    def normalizar_emociones_imagen(self, emociones_raw):
        emociones_norm = defaultdict(float)

        for emocion in emociones_raw:
            tipo = emocion['Type']
            confianza = emocion['Confidence']

            # Mapear a nombre común
            if tipo in self.mapeo_emociones and self.mapeo_emociones[tipo]:
                nombre_comun = self.mapeo_emociones[tipo]
                emociones_norm[nombre_comun] = confianza

        # Asegurar que todas las emociones comunes estén presentes
        for emocion in self.emociones_comunes:
            if emocion not in emociones_norm:
                emociones_norm[emocion] = 0.0

        return dict(emociones_norm)

    def normalizar_emociones_audio(self, emociones_raw):
        return {
            'Neutral': emociones_raw.get('Neutral', 0.0) * 100,  # Convertir a %
            'Felicidad': emociones_raw.get('Felicidad', 0.0) * 100,
            'Tristeza': emociones_raw.get('Tristeza', 0.0) * 100,
            'Ira': emociones_raw.get('Ira', 0.0) * 100,
            'Miedo': emociones_raw.get('Miedo', 0.0) * 100
        }

    def fusion_promedio_ponderado(self, emociones_imagen, emociones_audio,
                                  peso_imagen=0.5, peso_audio=0.5):
        resultado = {}
        for emocion in self.emociones_comunes:
            prob_imagen = emociones_imagen.get(emocion, 0.0)
            prob_audio = emociones_audio.get(emocion, 0.0)
            resultado[emocion] = (prob_imagen * peso_imagen + prob_audio * peso_audio)

        return resultado

    def calcular_confianza_fusion(self, emociones_imagen, emociones_audio):
        correlaciones = []
        for emocion in self.emociones_comunes:
            prob_imagen = emociones_imagen.get(emocion, 0.0)
            prob_audio = emociones_audio.get(emocion, 0.0)

            # Calcular diferencia normalizada
            diff = abs(prob_imagen - prob_audio) / 100.0
            concordancia = 1.0 - diff
            correlaciones.append(concordancia)

        # Confianza promedio
        confianza = np.mean(correlaciones)
        return max(0.0, min(1.0, confianza))

    def fusionar_emociones(self, emociones_rekognition, emociones_vokaturi,
                          peso_imagen=0.6, peso_audio=0.4):
        # Normalizar entradas
        img_norm = self.normalizar_emociones_imagen(emociones_rekognition)
        audio_norm = self.normalizar_emociones_audio(emociones_vokaturi)

        # Aplicar fusión ponderada
        resultado = self.fusion_promedio_ponderado(img_norm, audio_norm, peso_imagen, peso_audio)

        # Calcular confianza
        confianza = self.calcular_confianza_fusion(img_norm, audio_norm)

        # Ordenar resultados
        resultado_ordenado = sorted(resultado.items(), key=lambda x: x[1], reverse=True)

        return {
            'emociones_fusionadas': resultado_ordenado,
            'confianza_fusion': confianza,
            'emociones_imagen_normalizadas': img_norm,
            'emociones_audio_normalizadas': audio_norm
        }

In [5]:
# Crear cliente de Rekognition
rekognition = boto3.client(
    'rekognition',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

# Enviar solicitud para detectar emociones
response = rekognition.detect_faces(
    Image={'Bytes': image_bytes},
    Attributes=['ALL']
)

# Variables para almacenar resultados
emociones_imagen = None

# Mostrar emociones de imagen
if 'FaceDetails' in response and len(response['FaceDetails']) > 0:
    print("Emociones detectadas con Amazon Rekognition:")
    emotions = response['FaceDetails'][0]['Emotions']
    emociones_imagen = emotions  # Guardar para fusión
    for emotion in sorted(emotions, key=lambda x: x['Confidence'], reverse=True):
        print(f"{emotion['Type']}: {emotion['Confidence']:.2f}%")
else:
    print("No se detectaron rostros en la imagen.")

Emociones detectadas con Amazon Rekognition:
ANGRY: 100.00%
FEAR: 0.01%
DISGUSTED: 0.00%
CONFUSED: 0.00%
SURPRISED: 0.00%
SAD: 0.00%
HAPPY: 0.00%
CALM: 0.00%


In [6]:
zip_path = '/content/OpenVokaturi-4-0.zip'
if not os.path.exists(zip_path):
    print("Descargando Vokaturi...")
    !wget -O /content/OpenVokaturi-4-0.zip https://developers.vokaturi.com/downloads/OpenVokaturi-4-0.zip

# Descomprimir
if os.path.exists(zip_path):
    print(f"Descomprimiendo {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content')
    print("Vokaturi descomprimido correctamente")
else:
    print("Error: No se pudo descargar Vokaturi")

VOKATURI_API_PATH = "/content/OpenVokaturi-4-0/api"
VOKATURI_LIB_PATH = "/content/OpenVokaturi-4-0/lib/open/linux/OpenVokaturi-4-0-linux.so"

sys.path.append(VOKATURI_API_PATH)

# Verificar que existe y luego importar
if os.path.exists(VOKATURI_API_PATH + "/Vokaturi.py"):
    import Vokaturi
    print("Vokaturi importado exitosamente")
else:
    print("Error: No se encontró Vokaturi.py")
    print(f"Buscando en: {VOKATURI_API_PATH}")

Descomprimiendo /content/OpenVokaturi-4-0.zip...
Vokaturi descomprimido correctamente
Vokaturi importado exitosamente


In [7]:
def convertir_a_wav(archivo_entrada, archivo_salida):
    try:
        # Verificar si ffmpeg está instalado
        try:
            subprocess.run(['ffmpeg', '-version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except FileNotFoundError:
            print("Instalando ffmpeg...")
            subprocess.run(['apt-get', 'update', '-qq'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Convertir a WAV mono de 16-bit
        comando = [
            'ffmpeg',
            '-i', archivo_entrada,
            '-ac', '1',                # Mono
            '-ar', '44100',            # 44.1kHz
            '-acodec', 'pcm_s16le',    # 16-bit PCM
            '-y',                      # Sobrescribir
            archivo_salida
        ]

        resultado = subprocess.run(comando,
                                  stdout=subprocess.PIPE,
                                  stderr=subprocess.PIPE,
                                  text=True)

        if resultado.returncode != 0:
            print(f"Error al convertir: {resultado.stderr}")
            return False

        print(f"Archivo convertido a WAV")
        return True

    except Exception as e:
        print(f"Error en conversión: {str(e)}")
        return False

def analizar_emociones_audio(ruta_archivo):
    # Verificar el archivo
    if not os.path.exists(ruta_archivo):
        return {"error": f"Archivo no encontrado: {ruta_archivo}"}

    # Cargar la biblioteca
    try:
        Vokaturi.load(VOKATURI_LIB_PATH)
        print("Biblioteca Vokaturi cargada")
    except Exception as e:
        return {"error": f"Error cargando Vokaturi: {str(e)}"}

    try:
        # Leer el archivo WAV
        archivo = wave.open(ruta_archivo, "rb")

        # Obtener propiedades
        frecuencia = archivo.getframerate()
        muestras_total = archivo.getnframes()
        canales = archivo.getnchannels()
        duracion = muestras_total / frecuencia

        print(f"Audio: {duracion:.2f}s, {frecuencia}Hz, {canales} canal(es)")

        # Leer datos
        buffer = archivo.readframes(muestras_total)
        archivo.close()

        # Convertir a valores numéricos
        muestras = np.frombuffer(buffer, dtype=np.int16)

        # Convertir a mono si es estéreo
        if canales == 2:
            print("Convirtiendo estéreo a mono...")
            muestras = np.mean(muestras.reshape(-1, 2), axis=1).astype(np.int16)

        # Crear objeto Voice
        voz = Vokaturi.Voice(frecuencia, len(muestras), 1)

        # Convertir a formato C
        muestras_c = (ctypes.c_short * len(muestras))()
        for i in range(len(muestras)):
            muestras_c[i] = muestras[i]

        # Analizar
        voz.fill_int16array(len(muestras), muestras_c)

        # Objetos para resultados
        calidad = Vokaturi.Quality()
        emociones_prob = Vokaturi.EmotionProbabilities()

        # Extraer emociones
        voz.extract(calidad, emociones_prob)

        # Guardar resultados
        emociones = {
            "Neutral": emociones_prob.neutrality,
            "Felicidad": emociones_prob.happiness,
            "Tristeza": emociones_prob.sadness,
            "Ira": emociones_prob.anger,
            "Miedo": emociones_prob.fear
        }

        # Liberar recursos
        voz.destroy()

        return {"emociones": emociones}

    except Exception as e:
        print(f"Error en análisis: {str(e)}")
        import traceback
        traceback.print_exc()
        return {"error": str(e)}


In [8]:
emociones_audio = None

if audio_file:  # Solo proceder si se subió un archivo
    print("\nAnalizando el archivo de audio...")

    # Verificar si es WAV o MP3
    extension = os.path.splitext(audio_file)[1].lower()
    archivo_wav = audio_file

    # Convertir si es MP3
    if extension == '.mp3':
        print("Detectado archivo MP3, convirtiendo...")
        archivo_wav = audio_file.replace('.mp3', '.wav')
        if not convertir_a_wav(audio_file, archivo_wav):
            print("No se pudo convertir el archivo")
            archivo_wav = None
    elif extension != '.wav':
        print(f"Formato no soportado: {extension}")
        print("Solo se admiten archivos .wav o .mp3")
        archivo_wav = None

    # Analizar si tenemos un archivo WAV
    if archivo_wav:
        resultado = analizar_emociones_audio(archivo_wav)

        if isinstance(resultado, dict) and 'error' in resultado:
            print(f"Error: {resultado['error']}")
        else:
            # Guardar para fusión
            emociones_audio = resultado['emociones']

            # Mostrar resultados de audio
            emociones = {
                "NEUTRAL": resultado['emociones']['Neutral'] * 100,
                "HAPPY": resultado['emociones']['Felicidad'] * 100,
                "SAD": resultado['emociones']['Tristeza'] * 100,
                "ANGRY": resultado['emociones']['Ira'] * 100,
                "FEAR": resultado['emociones']['Miedo'] * 100
            }

            # Ordenar por valor
            emociones_ordenadas = sorted(emociones.items(), key=lambda x: x[1], reverse=True)

            print("\nEmociones detectadas con Vokaturi:")
            for emocion, valor in emociones_ordenadas:
                print(f"{emocion}: {valor:.2f}%")
else:
    print("No se subió ningún archivo de audio")


Analizando el archivo de audio...
Biblioteca Vokaturi cargada
Audio: 8.70s, 48000Hz, 1 canal(es)

Emociones detectadas con Vokaturi:
ANGRY: 99.87%
HAPPY: 0.13%
SAD: 0.00%
NEUTRAL: 0.00%
FEAR: 0.00%


In [9]:
print("\n" + "="*60)
print("FUSIONANDO RESULTADOS DE IMAGEN Y AUDIO")
print("="*60)

if emociones_imagen is not None and emociones_audio is not None:
    # Crear el fusionador de emociones
    fusionador = FusionEmociones()

    # Fusionar las emociones (60% peso imagen, 40% peso audio)
    resultado_fusion = fusionador.fusionar_emociones(
        emociones_imagen,
        emociones_audio,
        peso_imagen=0.6,
        peso_audio=0.4
    )

    # Mostrar resultados de la fusión
    print(f"\nConfianza de la fusión: {resultado_fusion['confianza_fusion']:.2%}")
    print("(Este valor indica qué tan concordantes son los resultados)")

    print("\n--- EMOCIONES DETECTADAS ---")
    print("\nPor IMAGEN (normalizado):")
    for emocion, valor in resultado_fusion['emociones_imagen_normalizadas'].items():
        print(f"  {emocion}: {valor:.2f}%")

    print("\nPor AUDIO (normalizado):")
    for emocion, valor in resultado_fusion['emociones_audio_normalizadas'].items():
        print(f"  {emocion}: {valor:.2f}%")

    print("\nRESULTADO FINAL (FUSIONADO):")
    print("Peso: 60% imagen + 40% audio")
    for emocion, valor in resultado_fusion['emociones_fusionadas']:
        print(f"  {emocion}: {valor:.2f}%")

    # Emoción principal
    if resultado_fusion['emociones_fusionadas']:
        emocion_principal = resultado_fusion['emociones_fusionadas'][0]
        print(f"\n EMOCIÓN DOMINANTE: {emocion_principal[0]} ({emocion_principal[1]:.2f}%)")

        # Interpretación de la confianza
        confianza = resultado_fusion['confianza_fusion']
        if confianza > 0.8:
            print("Alta concordancia entre imagen y audio")
        elif confianza > 0.6:
            print("Concordancia moderada entre imagen y audio")
        else:
            print("Baja concordancia entre imagen y audio - interpretar con cuidado")

else:
    if emociones_imagen is None and emociones_audio is None:
        print("No se pudieron analizar ni la imagen ni el audio")
    elif emociones_imagen is None:
        print("No se pudo analizar la imagen")
    else:
        print("No se pudo analizar el audio")
    print("No es posible realizar la fusión de emociones")

print("\n" + "="*60)
print("ANÁLISIS COMPLETADO")
print("="*60)


FUSIONANDO RESULTADOS DE IMAGEN Y AUDIO

Confianza de la fusión: 99.94%
(Este valor indica qué tan concordantes son los resultados)

--- EMOCIONES DETECTADAS ---

Por IMAGEN (normalizado):
  Ira: 100.00%
  Miedo: 0.01%
  Tristeza: 0.00%
  Felicidad: 0.00%
  Neutral: 0.00%

Por AUDIO (normalizado):
  Neutral: 0.00%
  Felicidad: 0.13%
  Tristeza: 0.00%
  Ira: 99.87%
  Miedo: 0.00%

RESULTADO FINAL (FUSIONADO):
Peso: 60% imagen + 40% audio
  Ira: 99.95%
  Felicidad: 0.05%
  Miedo: 0.01%
  Tristeza: 0.00%
  Neutral: 0.00%

 EMOCIÓN DOMINANTE: Ira (99.95%)
Alta concordancia entre imagen y audio

ANÁLISIS COMPLETADO
